# Points of Interest

## Imports and Setup

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append("../")

import logging
from pathlib import Path

from icecream import ic

from IPython.display import display, clear_output
import ipywidgets as wid
from utils.ipywidgets_extended import (
    widgets_styling,
    widgets_styling_slider,
    MultiSelect,
    RadioSelect,
)

from utils.setup_notebook import init_notebook
from utils.setup_logging import setup_logging
import utils.memoize as memoize

init_notebook()
setup_logging("INFO")
memoize.set_file_store_path("points_of_interest")

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import numba as nb
from numba import cuda
import cv2

from utils.benchmarking import LogTimer
from utils.plotting_tools import (
    SmartFigure,
    to_ipy_image,
    plot_kernel,
    plot_matrix,
)
from utils.image_tools import load_image, LoadedImage
import utils.dyn_module as dyn
from utils.cv2_tools import draw_keypoints, draw_matches
from utils.distinct_colors import bgrs

logging.getLogger("numba.cuda.cudadrv.driver").setLevel(logging.WARNING)

In [3]:
reset_memoize_store_button = wid.Button(description="Reset memoize store")
reset_memoize_store_button.on_click(lambda x: memoize.reset_store())
display(reset_memoize_store_button)


Button(description='Reset memoize store', style=ButtonStyle())

## Loading Points of Interest Implementations

In [4]:
dir_points_of_interest_impls = "./points_of_interest_impls"
points_of_interest_impls_module_names = dyn.load_modules(dir_points_of_interest_impls)


2024-11-10 17:59:03.266 INFO root Loading 1 modules started (..\utils\dyn_module.py:59)
2024-11-10 17:59:03.276 INFO root Loading harris_points_of_interest started (..\utils\dyn_module.py:32)
2024-11-10 17:59:03.293 INFO root Loading harris_points_of_interest took: 17.6725 ms (..\utils\dyn_module.py:32)
2024-11-10 17:59:03.309 INFO root Loading 1 modules took: 299.6965 ms (..\utils\dyn_module.py:59)


## Loading Input Image Sets

In [5]:
input_image_set_dir = "./image_sets_input"
input_image_set_scaled_dir = "./image_sets_scaled_input"

os.makedirs(input_image_set_dir, exist_ok=True)
os.makedirs(input_image_set_scaled_dir, exist_ok=True)

input_image_sets = {}


def load_image_sets(image_sets_dir: str):
    image_sets_folders = os.listdir(image_sets_dir)
    image_sets_folders.sort()

    with LogTimer(f"Loading image sets from {Path(image_sets_dir).name}"):
        for image_set_name in image_sets_folders:
            with LogTimer(f"Loading image set {image_set_name}"):
                image_set_dir = os.path.join(image_sets_dir, image_set_name)
                image_set = []

                images_in_image_set = os.listdir(image_set_dir)
                images_in_image_set.sort()

                for image_name in images_in_image_set:
                    image = load_image(os.path.join(image_set_dir, image_name))
                    image_set.append(image)

                input_image_sets[image_set_name] = image_set


def save_image_set(image_set: list, image_set_dir: str):
    os.makedirs(image_set_dir, exist_ok=True)
    for image in image_set:
        # if not jpg or png set to png
        filepath = Path(image.filename)
        if filepath.suffix not in [".jpg", ".png"]:
            output_filename = filepath.stem + ".png"
        else:
            output_filename = filepath.name
        cv2.imwrite(os.path.join(image_set_dir, output_filename), image.image_color)


load_image_sets(input_image_set_dir)
load_image_sets(input_image_set_scaled_dir)

2024-11-10 17:59:03.416 INFO root Loading image sets from image_sets_input started (notebook_cell:14)
2024-11-10 17:59:03.433 INFO root Loading image set objects_sweets started (notebook_cell:16)
2024-11-10 17:59:03.450 INFO root Loading object_bisasam.jpg started (..\utils\image_tools.py:69)
2024-11-10 17:59:03.534 INFO root Loading object_bisasam.jpg took: 70.3901 ms (..\utils\image_tools.py:69)
2024-11-10 17:59:03.558 INFO root Loading object_choco_back.tiff started (..\utils\image_tools.py:69)
2024-11-10 17:59:03.724 INFO root Loading object_choco_back.tiff took: 169.5325 ms (..\utils\image_tools.py:69)
2024-11-10 17:59:03.731 INFO root Loading object_choco_front.tiff started (..\utils\image_tools.py:69)
2024-11-10 17:59:03.917 INFO root Loading object_choco_front.tiff took: 172.0131 ms (..\utils\image_tools.py:69)
2024-11-10 17:59:03.934 INFO root Loading object_green_back.tiff started (..\utils\image_tools.py:69)
2024-11-10 17:59:04.176 INFO root Loading object_green_back.tiff to

## Image set scaler

In [6]:
def get_largest_image_in_image_set(image_set: list) -> np.array:
    largest_image = None
    largest_pixel_count = 0
    for image in image_set:
        resolution = image.image_color.shape[:2]
        pixel_count = resolution[0] * resolution[1]
        if pixel_count > largest_pixel_count:
            largest_pixel_count = pixel_count
            largest_image = image
    return largest_image


def get_largest_resolution_in_image_set(image_set: list) -> tuple:
    largest_image = get_largest_image_in_image_set(image_set)
    return largest_image.image_color.shape[:2]

In [7]:
def scaler():
    clear_output()
    KEY_SCALER_IMAGE_SET_DROPDOWN = "scaler_image_set_dropdown"
    scaler_image_set_dropdown = wid.Dropdown(
        options=list(input_image_sets.keys()),
        value=memoize.get(
            KEY_SCALER_IMAGE_SET_DROPDOWN,
            default=next(iter(input_image_sets.keys())),
            possible_values=input_image_sets.keys(),
        ),
        description="Image set",
        **widgets_styling,
    )
    scaler_largest_resolution_label = wid.Label("Largest image: (X,X)")
    KEY_SCALER_SCALE_SLIDER = "scaler_scale_slider"
    scaler_scale_slider = wid.FloatSlider(
        value=memoize.get(KEY_SCALER_SCALE_SLIDER, default=1.0),
        min=0.1,
        max=30.0,
        step=0.1,
        continuous_update=True,
        orientation="horizontal",
        readout=True,
        readout_format=".1f",
        description="Scale",
        **widgets_styling_slider,
    )
    scaler_result_resolution_label = wid.Label("Resulting largest image: (X,X)")
    scaler_create_scaled_image_set_button = wid.Button(
        description="Create scaled image set",
        **widgets_styling,
    )

    def on_update_resolution_labels(change=None):
        memoize.set(KEY_SCALER_IMAGE_SET_DROPDOWN, scaler_image_set_dropdown.value)
        memoize.set(KEY_SCALER_SCALE_SLIDER, scaler_scale_slider.value)

        image_set = input_image_sets[scaler_image_set_dropdown.value]
        largest_image = get_largest_image_in_image_set(image_set)
        largest_resolution = largest_image.image_color.shape[:2]
        scaler_largest_resolution_label.value = f"Largest image: {largest_resolution}"
        scaler_result_resolution_label.value = f"Resulting largest image: {tuple(int(x * 1/scaler_scale_slider.value) for x in largest_resolution)}"

    scaler_scale_slider.observe(on_update_resolution_labels, names="value")
    scaler_image_set_dropdown.observe(on_update_resolution_labels, names="value")
    on_update_resolution_labels()

    def create_scaled_image_set(change=None):
        with LogTimer(
            f"Creating scaled image set for {scaler_image_set_dropdown.value}"
        ):
            original_image_set = input_image_sets[scaler_image_set_dropdown.value]

            scale = 1 / scaler_scale_slider.value

            largest_resolution = get_largest_resolution_in_image_set(original_image_set)
            scaled_resolution = tuple(int(x * scale) for x in largest_resolution)
            scaled_resolution_fs_string = (
                f"({scaled_resolution[0]},{scaled_resolution[1]})"
            )

            scaled_image_set_name = f"{scaler_image_set_dropdown.value}_scaled_{scaled_resolution_fs_string}"

            scaled_image_set_dir = os.path.join(
                input_image_set_scaled_dir, scaled_image_set_name
            )

            new_image_set = []
            for image in original_image_set:
                scaled_resolution = tuple(
                    int(x * scale) for x in image.image_color.shape[:2]
                )
                scaled_resolution_fs_string = (
                    f"({scaled_resolution[0]},{scaled_resolution[1]})"
                )
                with LogTimer(
                    f"Resizing image {image.filename} from {image.image_color.shape[:2]} to {scaled_resolution}"
                ):
                    new_image = LoadedImage()
                    new_image.image_color = cv2.resize(
                        image.image_color, (scaled_resolution[1], scaled_resolution[0])
                    )
                    new_image.filename = f"{Path(image.filename).stem}_{scaled_resolution_fs_string}{Path(image.filename).suffix}"
                    new_image_set.append(new_image)

            save_image_set(new_image_set, scaled_image_set_dir)
            load_image_sets(input_image_set_scaled_dir)
            scaler()

    scaler_create_scaled_image_set_button.on_click(create_scaled_image_set)

    display(
        wid.VBox(
            [
                wid.HBox([scaler_image_set_dropdown, scaler_largest_resolution_label]),
                wid.HBox([scaler_scale_slider, scaler_result_resolution_label]),
                scaler_create_scaled_image_set_button,
            ]
        )
    )


scaler()

## Running Harris Points of Interest detection and Image stitching

In [8]:
fig_width = 16
image_size = 512

# filter keys by "stitch"
input_image_sets_stitch = {
    key: value for key, value in input_image_sets.items() if "stitch" in key
}

KEY_IMAGE_SET_DROPDOWN = "image_set_dropdown"
image_set_dropdown = wid.Dropdown(
    options=list(input_image_sets_stitch.keys()),
    value=memoize.get(
        KEY_IMAGE_SET_DROPDOWN,
        default=next(iter(input_image_sets_stitch.keys())),
        possible_values=input_image_sets_stitch.keys(),
    ),
    description="Image set",
    **widgets_styling,
)
KEY_POINTS_OF_INTEREST_IMPL_DROPDOWN = "points_of_interest_impl_dropdown"
points_of_interest_impl_dropdown = wid.Dropdown(
    options=points_of_interest_impls_module_names,
    value=memoize.get(
        KEY_POINTS_OF_INTEREST_IMPL_DROPDOWN,
        default=points_of_interest_impls_module_names[0],
        possible_values=points_of_interest_impls_module_names,
    ),
    description="Point of interest implementation",
    **widgets_styling,
)
reload_impl_button = wid.Button(
    description="Reload Implementation",
    **widgets_styling,
)
output = wid.Output()


@output.capture(clear_output=True, wait=True)
def on_menu_change(change=None):
    memoize.set(KEY_IMAGE_SET_DROPDOWN, image_set_dropdown.value)
    memoize.set(
        KEY_POINTS_OF_INTEREST_IMPL_DROPDOWN, points_of_interest_impl_dropdown.value
    )

    # reload the impl module
    current_points_of_interest_impl = points_of_interest_impl_dropdown.value
    points_of_interest_impl = dyn.load_module(current_points_of_interest_impl)

    input_image_set = input_image_sets_stitch[image_set_dropdown.value]
    total_image_count = len(input_image_set)
    KEY_IMAGE_COUNT_SLIDER = f"image_count_slider_{image_set_dropdown.value}"
    current_image_count_slider_value = memoize.get(
        KEY_IMAGE_COUNT_SLIDER, default=total_image_count
    )
    if current_image_count_slider_value > total_image_count:
        current_image_count_slider_value = total_image_count
    image_count_slider = wid.IntSlider(
        value=current_image_count_slider_value,
        min=1,
        max=total_image_count,
        step=1,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format="d",
        description="Image count",
        **widgets_styling_slider,
    )

    def on_image_count_slider_change(change=None):
        memoize.set(KEY_IMAGE_COUNT_SLIDER, image_count_slider.value)
        on_menu_change()

    image_count_slider.observe(on_image_count_slider_change, names="value")
    display(image_count_slider)

    image_set = input_image_set[: image_count_slider.value]

    with LogTimer("Displaying input images"):
        input_ipy_images_color = [
            to_ipy_image(image.image_color, longest_side=image_size, upscale=True)
            for image in image_set
        ]
        display(wid.HBox(input_ipy_images_color))
        input_ipy_images_gray = [
            to_ipy_image(image.image_gray, longest_side=image_size, upscale=True)
            for image in image_set
        ]
        display(wid.HBox(input_ipy_images_gray))

    KEY_SIGMA1_SLIDER = "sigma1_slider"
    sigma1_slider = wid.FloatSlider(
        value=memoize.get(KEY_SIGMA1_SLIDER, default=0.8),
        min=0.1,
        max=20.0,
        step=0.1,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".1f",
        description="Sigma 1",
        **widgets_styling_slider,
    )
    KEY_SIGMA2_SLIDER = "sigma2_slider"
    sigma2_slider = wid.FloatSlider(
        value=memoize.get(KEY_SIGMA2_SLIDER, default=1.5),
        min=0.1,
        max=20.0,
        step=0.1,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".1f",
        description="Sigma 2",
        **widgets_styling_slider,
    )
    KEY_THRESHOLD_SLIDER = "threshold_slider"
    threshold_slider = wid.FloatSlider(
        value=memoize.get(KEY_THRESHOLD_SLIDER, default=0.01),
        min=0.0,
        max=0.1,
        step=0.01,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".2f",
        description="Threshold",
        **widgets_styling_slider,
    )
    KEY_HARRIS_K_SLIDER = "harris_k_slider"
    harris_k_slider = wid.FloatSlider(
        value=memoize.get(KEY_HARRIS_K_SLIDER, default=0.04),
        min=0.01,
        max=0.1,
        step=0.01,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".2f",
        description="Harris k",
        **widgets_styling_slider,
    )
    default_harris_button = wid.Button(
        description="Default Harris",
        **widgets_styling,
    )
    output_harris_corner = wid.Output()

    @output_harris_corner.capture(clear_output=True, wait=True)
    def on_harris_change(config=None):
        memoize.set(KEY_SIGMA1_SLIDER, sigma1_slider.value)
        memoize.set(KEY_SIGMA2_SLIDER, sigma2_slider.value)
        memoize.set(KEY_THRESHOLD_SLIDER, threshold_slider.value)
        memoize.set(KEY_HARRIS_K_SLIDER, harris_k_slider.value)

        image_gray_array = np.stack([image.image_gray for image in image_set])
        image_color_array = np.stack([image.image_color for image in image_set])

        with LogTimer("Calculating Harris corners"):
            harris_corner_keystones = points_of_interest_impl.harris_corner(
                image_gray_array,
                sigma1_slider.value,
                sigma2_slider.value,
                harris_k_slider.value,
                threshold_slider.value,
            )

        with LogTimer("Displaying Harris corners"):
            annotated_harris_ipy_images = []
            for image_idx, image in enumerate(image_set):
                keypoint_size = int(max(image.image_color.shape[:2]) / 512)
                for keypoint in harris_corner_keystones[image_idx]:
                    keypoint.size *= keypoint_size

                annotated_image = draw_keypoints(
                    image_color_array[image_idx],
                    harris_corner_keystones[image_idx],
                )
                annotated_harris_ipy_images.append(
                    wid.VBox(
                        [
                            wid.Label(f"{image.filename}"),
                            wid.Label(
                                f"Found {len(harris_corner_keystones[image_idx])} Harris corners"
                            ),
                            to_ipy_image(
                                annotated_image, longest_side=image_size, upscale=True
                            ),
                        ]
                    )
                )
            display(wid.HTML("<h2>Harris corners</h2>"))
            display(wid.HBox(annotated_harris_ipy_images))

            if total_image_count < 2:
                display(wid.HTML("<h2>Not enough images to calculate matches</h2>"))
                return
            # Flann based matcher
            display(wid.HTML("<h2>Flann matches</h2>"))
            KEY_FLANN_IMAGE_1_DROPDOWN = "flann_image_1_dropdown"
            flann_image_1_dropdown = wid.Dropdown(
                options=[image.filename for image in image_set],
                value=memoize.get(
                    KEY_FLANN_IMAGE_1_DROPDOWN,
                    default=image_set[0].filename,
                    possible_values=[image.filename for image in image_set],
                ),
                description="Image 1",
                **widgets_styling,
            )
            KEY_FLANN_IMAGE_2_DROPDOWN = "flann_image_2_dropdown"
            flann_image_2_dropdown = wid.Dropdown(
                options=[image.filename for image in image_set],
                value=memoize.get(
                    KEY_FLANN_IMAGE_2_DROPDOWN,
                    default=image_set[1].filename,
                    possible_values=[image.filename for image in image_set],
                ),
                description="Image 2",
                **widgets_styling,
            )
            KEY_PATCH_SIZE_SLIDER = "patch_size_slider"
            patch_size_slider = wid.IntSlider(
                value=memoize.get(KEY_PATCH_SIZE_SLIDER, default=5),
                min=1,
                max=15,
                step=1,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                readout_format="d",
                description="Patch size",
                **widgets_styling_slider,
            )
            default_flann_button = wid.Button(
                description="Default flann values",
                **widgets_styling,
            )
            output_flann = wid.Output()

            @output_flann.capture(clear_output=True, wait=True)
            def on_flann_change(config=None):
                memoize.set(KEY_FLANN_IMAGE_1_DROPDOWN, flann_image_1_dropdown.value)
                memoize.set(KEY_FLANN_IMAGE_2_DROPDOWN, flann_image_2_dropdown.value)
                memoize.set(KEY_PATCH_SIZE_SLIDER, patch_size_slider.value)

                image_1_idx = [image.filename for image in image_set].index(
                    flann_image_1_dropdown.value
                )
                image_2_idx = [image.filename for image in image_set].index(
                    flann_image_2_dropdown.value
                )

                image_gray_1 = image_gray_array[image_1_idx]
                image_gray_2 = image_gray_array[image_2_idx]

                keypoints_1 = harris_corner_keystones[image_1_idx]
                keypoints_2 = harris_corner_keystones[image_2_idx]

                with LogTimer("Compute Descriptors"):
                    filtered_keypoints_1, descriptors_1 = (
                        points_of_interest_impl.compute_descriptors(
                            image_gray_1, keypoints_1, patch_size_slider.value
                        )
                    )
                    filtered_keypoints_2, descriptors_2 = (
                        points_of_interest_impl.compute_descriptors(
                            image_gray_2, keypoints_2, patch_size_slider.value
                        )
                    )

                with LogTimer("Calculating Flann matches"):
                    matches = points_of_interest_impl.flann_matches(
                        descriptors_1, descriptors_2
                    )

                with LogTimer("Filtering Flann matches"):
                    matches_filtered = points_of_interest_impl.filter_matches(matches)

                display(
                    f"Found {len(matches)} matches. These were filtered down to {len(matches_filtered)} matches ({len(matches_filtered)/len(matches)*100:.2f}%)"
                )

                with LogTimer("Drawing Flann matches"):
                    draw_matches_image = draw_matches(
                        image_color_array[image_1_idx],
                        filtered_keypoints_1,
                        image_color_array[image_2_idx],
                        filtered_keypoints_2,
                        matches_filtered,
                    )

                with LogTimer("Displaying Flann matches"):
                    display(
                        to_ipy_image(
                            draw_matches_image,
                            longest_side=image_size * 3,
                            upscale=True,
                        )
                    )

            flann_image_1_dropdown.observe(on_flann_change, names="value")
            flann_image_2_dropdown.observe(on_flann_change, names="value")
            patch_size_slider.observe(on_flann_change, names="value")

            def default_flann(change=None):
                memoize.delete_keys(
                    [
                        KEY_FLANN_IMAGE_1_DROPDOWN,
                        KEY_FLANN_IMAGE_2_DROPDOWN,
                        KEY_PATCH_SIZE_SLIDER,
                    ]
                )
                on_menu_change()

            default_flann_button.on_click(default_flann)

            display(
                wid.VBox(
                    [
                        wid.HBox([flann_image_1_dropdown, flann_image_2_dropdown]),
                        wid.HBox([patch_size_slider, default_flann_button]),
                        output_flann,
                    ]
                )
            )
            on_flann_change()

    sigma1_slider.observe(on_harris_change, names="value")
    sigma2_slider.observe(on_harris_change, names="value")
    threshold_slider.observe(on_harris_change, names="value")
    harris_k_slider.observe(on_harris_change, names="value")

    def default_harris(change=None):
        memoize.delete_keys(
            [
                KEY_SIGMA1_SLIDER,
                KEY_SIGMA2_SLIDER,
                KEY_THRESHOLD_SLIDER,
                KEY_HARRIS_K_SLIDER,
            ]
        )
        on_menu_change()

    default_harris_button.on_click(default_harris)

    display(
        wid.VBox(
            [
                sigma1_slider,
                sigma2_slider,
                threshold_slider,
                harris_k_slider,
                default_harris_button,
                output_harris_corner,
            ]
        )
    )
    on_harris_change()


image_set_dropdown.observe(on_menu_change, names="value")
points_of_interest_impl_dropdown.observe(on_menu_change, names="value")
reload_impl_button.on_click(on_menu_change)

display(
    wid.VBox(
        [
            wid.HBox(
                [
                    image_set_dropdown,
                    points_of_interest_impl_dropdown,
                    reload_impl_button,
                ]
            ),
            output,
        ]
    )
)
on_menu_change()

## Least Squares Homography Debug

In [9]:
# TODO: Allow other homography implementations to be selected

KEY_HOMOGRAPHY_D_POINTS_OF_INTEREST_IMPL_DROPDOWN = (
    "homography_d_points_of_interest_impl_dropdown"
)
homography_d_points_of_interest_impl_dropdown = wid.Dropdown(
    options=points_of_interest_impls_module_names,
    value=memoize.get(
        KEY_HOMOGRAPHY_D_POINTS_OF_INTEREST_IMPL_DROPDOWN,
        default=points_of_interest_impls_module_names[0],
        possible_values=points_of_interest_impls_module_names,
    ),
    description="Point of interest implementation",
    **widgets_styling,
)
KEY_HOMOGRAPHY_D_USE_SVD_CHECKBOX = "homography_d_use_svd_checkbox"
homography_d_use_svd_checkbox = wid.Checkbox(
    value=memoize.get(KEY_HOMOGRAPHY_D_USE_SVD_CHECKBOX, default=True),
    description="Use SVD",
    **widgets_styling,
)
homography_d_reload_impl_button = wid.Button(
    description="Reload Implementation",
    **widgets_styling,
)
homography_d_output = wid.Output()

color_magenta = tuple([int(x / 2) for x in (255, 0, 255)])


def draw_center_lines(image_io: np.ndarray) -> np.ndarray:
    height, width = image_io.shape[:2]
    cv2.line(image_io, (width // 2, 0), (width // 2, height), (255, 0, 0), 2)
    cv2.line(image_io, (0, height // 2), (width, height // 2), (255, 0, 0), 2)


def image_to_verts(image_i: np.ndarray) -> np.ndarray:
    height, width = image_i.shape[:2]
    return np.array(
        [
            [0, 0],
            [width, 0],
            [width, height],
            [0, height],
        ],
        dtype=np.int32,
    )


def draw_verts(
    image_io: np.ndarray,
    verts_i: np.ndarray,
    thickness=10,
    fill_color=None,
    outline_color=None,
) -> np.ndarray:
    if fill_color is not None:
        cv2.fillPoly(image_io, [verts_i], fill_color)
    if outline_color is None:
        line_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]
    else:
        line_colors = [outline_color] * 4
    for i in range(4):
        cv2.line(
            image_io,
            tuple(verts_i[i]),
            tuple(verts_i[(i + 1) % 4]),
            color=line_colors[i],
            thickness=thickness,
        )


def random_rectangle_transform(
    image_i: np.ndarray, rectangle_verts_i: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Create a random rectangle in the image and return its vertices in the image and in the object space.

    :return: (generated_image, target_points, source_points):
        generated_image: image with a single projected rectangle
        target_points: array of keypoints_1 in the target image in the shape (n, 2)
        source_points: array of keypoints_1 in the source image in the shape (n, 2)
    :rtype: (np.ndarray, np.ndarray, np.ndarray)
    """

    image_height, image_width = image_i.shape[:2]

    # Compute the height and width of the rectangle from its vertices
    min_coords = np.min(rectangle_verts_i, axis=0)
    max_coords = np.max(rectangle_verts_i, axis=0)
    rect_width = max_coords[0] - min_coords[0]
    rect_height = max_coords[1] - min_coords[1]

    # Move the rectangle to the center of the scene
    image_rectangle_verts = (
        rectangle_verts_i
        + [
            image_width / 2.0 - rect_width / 2.0,
            image_height / 2.0 - rect_height / 2.0,
        ]
        + np.around(10 * np.random.randn(4, 2))
    )
    image_rectangle_verts = image_rectangle_verts.astype(np.int32)

    return image_rectangle_verts


def draw_projected_object(
    image_io: np.ndarray,
    object_image_i: np.ndarray,
    homography_i: np.ndarray,
    draw_object=True,
    fill_color=color_magenta,
    outline_color=None,
) -> np.ndarray:
    # Get the vertices of the object image
    object_verts_i = image_to_verts(object_image_i).astype(np.float32)

    # Project the object image to the input image
    # We have to reshape because openCV expects batched input
    projected_verts_i = (
        cv2.perspectiveTransform(object_verts_i.reshape(1, -1, 2), homography_i)
        .reshape(-1, 2)
        .astype(np.int32)
    )

    draw_verts(
        image_io,
        projected_verts_i,
        thickness=2,
        fill_color=fill_color,
        outline_color=outline_color,
    )

    if draw_object:
        # Draw the object image in the projected position
        object_image_p = cv2.warpPerspective(
            object_image_i,
            homography_i,
            (image_io.shape[1], image_io.shape[0]),
        )

        cv2.copyTo(src=object_image_p, mask=object_image_p, dst=image_io)


@homography_d_output.capture(clear_output=True, wait=True)
def on_homography_d_menu_change(change=None):
    memoize.set(
        KEY_HOMOGRAPHY_D_POINTS_OF_INTEREST_IMPL_DROPDOWN,
        homography_d_points_of_interest_impl_dropdown.value,
    )
    impl_name = homography_d_points_of_interest_impl_dropdown.value
    points_of_interest_impl = dyn.load_module(impl_name)

    test_scene_image_height, test_scene_width = 320, 480
    test_scene_image = np.zeros(
        shape=(test_scene_image_height, test_scene_width, 3),
        dtype=np.uint8,
    )

    object_image = np.full(
        shape=(120, 200, 3), dtype=np.uint8, fill_value=color_magenta
    )
    draw_center_lines(object_image)
    object_verts = image_to_verts(object_image)

    # draw_center_lines(test_scene_image)

    image_rectangle_verts = random_rectangle_transform(test_scene_image, object_verts)
    draw_verts(test_scene_image, image_rectangle_verts)

    display(to_ipy_image(test_scene_image, longest_side=image_size, upscale=True))

    with LogTimer("Calculating homography"):
        homography = points_of_interest_impl.find_homography_eq(
            object_verts,
            image_rectangle_verts,
            use_svd=homography_d_use_svd_checkbox.value,
        ).homography

    draw_projected_object(test_scene_image, object_image, homography)
    display(to_ipy_image(test_scene_image, longest_side=image_size, upscale=True))


homography_d_points_of_interest_impl_dropdown.observe(
    on_homography_d_menu_change, names="value"
)
homography_d_use_svd_checkbox.observe(on_homography_d_menu_change, names="value")
homography_d_reload_impl_button.on_click(on_homography_d_menu_change)

display(
    wid.VBox(
        [
            wid.HBox(
                [
                    homography_d_points_of_interest_impl_dropdown,
                    homography_d_use_svd_checkbox,
                    homography_d_reload_impl_button,
                ]
            ),
            homography_d_output,
        ]
    )
)
on_homography_d_menu_change()

## Find objects using RANSAC and SIFT and Homography

In [ ]:
# filter keys by "objects"
input_image_sets_objects = {
    key: value for key, value in input_image_sets.items() if "objects" in key
}

KEY_FINDER_IMAGE_SET_DROPDOWN = "finder_image_set_dropdown"
finder_image_set_dropdown = wid.Dropdown(
    options=list(input_image_sets_objects.keys()),
    value=memoize.get(
        KEY_FINDER_IMAGE_SET_DROPDOWN,
        default=next(iter(input_image_sets_objects.keys())),
        possible_values=input_image_sets_objects.keys(),
    ),
    description="Image set",
    **widgets_styling,
)
KEY_FINDER_POINTS_OF_INTEREST_IMPL_DROPDOWN = "finder_points_of_interest_impl_dropdown"
finder_points_of_interest_impl_dropdown = wid.Dropdown(
    options=points_of_interest_impls_module_names,
    value=memoize.get(
        KEY_FINDER_POINTS_OF_INTEREST_IMPL_DROPDOWN,
        default=points_of_interest_impls_module_names[0],
        possible_values=points_of_interest_impls_module_names,
    ),
    description="Point of interest implementation",
    **widgets_styling,
)
finder_reload_impl_button = wid.Button(
    description="Reload Implementation",
    **widgets_styling,
)
finder_output = wid.Output()


@finder_output.capture(clear_output=True, wait=False)
def on_finder_menu_change(change=None):
    memoize.set(KEY_FINDER_IMAGE_SET_DROPDOWN, finder_image_set_dropdown.value)
    memoize.set(
        KEY_FINDER_POINTS_OF_INTEREST_IMPL_DROPDOWN,
        finder_points_of_interest_impl_dropdown.value,
    )

    # reload the impl module
    current_points_of_interest_impl = finder_points_of_interest_impl_dropdown.value
    points_of_interest_impl = dyn.load_module(current_points_of_interest_impl)

    color_it = bgrs()

    # extract scene images from the set
    with LogTimer("Loading scene images"):
        scene_images = [
            image
            for image in input_image_sets_objects[finder_image_set_dropdown.value]
            if "scene" in image.filename
        ]
        scene_image_wids = [
            to_ipy_image(
                image.image_color,
                longest_side=image_size / 3,
                upscale=True,
                set_dimensions=True,
            )
            for image in scene_images
        ]
        scene_images_filenames = [image.filename for image in scene_images]

    KEY_FINDER_SCENE_IMAGE_SELECT = "finder_scene_image_select"
    finder_scene_image_select = RadioSelect(
        all_choices=scene_images_filenames,
        default_choice=memoize.get(
            KEY_FINDER_SCENE_IMAGE_SELECT,
            default=scene_images_filenames[0],
            possible_values=scene_images_filenames,
        ),
        custom_widgets=scene_image_wids,
        grid_template_columns="1fr 1fr 1fr 1fr 1fr",
    )

    # extract object images from the set
    with LogTimer("Loading object images"):
        object_images = [
            image
            for image in input_image_sets_objects[finder_image_set_dropdown.value]
            if "object" in image.filename
        ]
        object_colors = [next(color_it) for _ in object_images]
        # multiply the color by 255 to get the color in the range 0-255
        object_colors = [
            (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)) for x in object_colors
        ]
        border_width = 10
        framed_object_images = [
            cv2.copyMakeBorder(
                image.image_color,
                border_width,
                border_width,
                border_width,
                border_width,
                cv2.BORDER_CONSTANT,
                value=color,
            )
            for image, color in zip(object_images, object_colors)
        ]
        object_image_wids = [
            to_ipy_image(
                image,
                longest_side=image_size / 3,
                upscale=True,
                set_dimensions=True,
            )
            for image in framed_object_images
        ]
        object_images_filenames = [image.filename for image in object_images]

    KEY_FINDER_OBJECT_IMAGE_SELECT = "finder_object_image_select1"
    finder_object_image_select = MultiSelect(
        all_choices=object_images_filenames,
        default_choices=memoize.get(
            KEY_FINDER_OBJECT_IMAGE_SELECT,
            default=object_images_filenames,
            possible_values=object_images_filenames,
            multi_value=True,
        ),
        custom_widgets=object_image_wids,
        grid_template_columns="1fr 1fr 1fr 1fr 1fr",
    )

    finder_image_selection_output = wid.Output()

    @finder_image_selection_output.capture(clear_output=True, wait=False)
    def on_finder_image_selection_change(change=None):
        memoize.set(
            KEY_FINDER_OBJECT_IMAGE_SELECT, finder_object_image_select.get_selected()
        )
        memoize.set(
            KEY_FINDER_SCENE_IMAGE_SELECT, finder_scene_image_select.get_selected()
        )

        scene_image_selected = scene_images[
            scene_images_filenames.index(finder_scene_image_select.get_selected())
        ]
        scene_image_selected_gray = scene_image_selected.image_gray

        object_images_selected_indices = [
            object_images_filenames.index(filename)
            for filename in finder_object_image_select.get_selected()
        ]
        object_images_selected = [
            object_images[index] for index in object_images_selected_indices
        ]
        object_images_selected_gray = [
            image.image_gray for image in object_images_selected
        ]

        with LogTimer("Running sift"):
            object_recognition_result = points_of_interest_impl.run_object_recognition(
                object_images_selected_gray, [scene_image_selected_gray]
            )

        object_images_selected_filenames = [
            image.filename for image in object_images_selected
        ]

        # Show the sift result
        if len(object_images_selected) != 0:
            KEY_FINDER_CMP_OBJECT_IMAGE_DROPDOWN = "finder_cmp_object_image_dropdown"
            finder_cmp_object_image_dropdown = wid.Dropdown(
                options=object_images_selected_filenames,
                value=memoize.get(
                    KEY_FINDER_CMP_OBJECT_IMAGE_DROPDOWN,
                    default=object_images_selected_filenames[0],
                    possible_values=object_images_selected_filenames,
                ),
                description="Object image",
                **widgets_styling,
            )
            finder_cmp_output = wid.Output()
            finder_cmp_output_workaround = wid.VBox([])

            # Temporary capturing is bugged in vscode
            # @finder_cmp_output.capture(
            #    clear_output=True, wait=False
            # )
            def on_finder_cmp_change(change=None):
                memoize.set(
                    KEY_FINDER_CMP_OBJECT_IMAGE_DROPDOWN,
                    finder_cmp_object_image_dropdown.value,
                )

                selected_object_image_index = object_images_selected_filenames.index(
                    finder_cmp_object_image_dropdown.value
                )

                selected_object_image_color = object_images_selected[
                    selected_object_image_index
                ].image_color

                selected_object_keypoints = (
                    object_recognition_result.detected_objects_keypoints[
                        selected_object_image_index
                    ]
                )
                selected_scene_keypoints = (
                    object_recognition_result.detected_scenes_keypoints[0]
                )
                selected_matches = object_recognition_result.object_scene_matches[
                    selected_object_image_index
                ][0]

                with LogTimer("Displaying sift result"):
                    annotated_image = draw_matches(
                        image_1_i=selected_object_image_color,
                        keypoints_1=selected_object_keypoints,
                        image_2_i=scene_image_selected.image_color,
                        keypoints_2=selected_scene_keypoints,
                        matches=selected_matches,
                    )
                    ipy_image = to_ipy_image(
                        annotated_image,
                        longest_side=image_size * 3,
                        upscale=True,
                    )
                    finder_cmp_output_workaround.children = [ipy_image]
                    # sadly bugged on vscode display(ipy_image)

            finder_cmp_object_image_dropdown.observe(
                on_finder_cmp_change, names="value"
            )

            display(
                wid.VBox(
                    [
                        finder_cmp_object_image_dropdown,
                        finder_cmp_output_workaround,
                        finder_cmp_output,
                    ]
                )
            )
            on_finder_cmp_change()

        # RUN RANSAC
        KEY_RANSAC_CONFIDENCE_SLIDER = "ransac_confidence_slider"
        ransac_confidence_slider = wid.FloatSlider(
            value=memoize.get(KEY_RANSAC_CONFIDENCE_SLIDER, default=0.85),
            min=0.1,
            max=0.99,
            step=0.01,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format=".2f",
            description="RANSAC confidence",
            **widgets_styling_slider,
        )
        KEY_RANSAC_INLIER_THRESHOLD_SLIDER = "ransac_inlier_threshold_slider"
        ransac_inlier_threshold_slider = wid.FloatSlider(
            value=memoize.get(KEY_RANSAC_INLIER_THRESHOLD_SLIDER, default=5.0),
            min=0.1,
            max=10.0,
            step=0.1,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format=".1f",
            description="RANSAC inlier threshold",
            **widgets_styling_slider,
        )
        ransac_default_button = wid.Button(
            description="Default RANSAC",
            **widgets_styling,
        )
        ransac_output = wid.Output()

        @ransac_output.capture(clear_output=True, wait=False)
        def on_ransac_change(change=None):
            memoize.set(KEY_RANSAC_CONFIDENCE_SLIDER, ransac_confidence_slider.value)
            memoize.set(
                KEY_RANSAC_INLIER_THRESHOLD_SLIDER, ransac_inlier_threshold_slider.value
            )

            selected_scene_keypoints = (
                object_recognition_result.detected_scenes_keypoints[0]
            )
            selected_objects_keypoints = [
                object_recognition_result.detected_objects_keypoints[i]
                for i, _ in enumerate(object_images_selected)
            ]

            selected_objects_matches = [
                object_recognition_result.object_scene_matches[i][0]
                for i, _ in enumerate(object_images_selected)
            ]

            with LogTimer("Running find homography"):
                homography_results = []
                for selected_object_image_index in range(
                    len(object_images_selected_indices)
                ):
                    selected_object_name = object_images_selected_filenames[
                        selected_object_image_index
                    ]

                    selected_object_keypoints = selected_objects_keypoints[
                        selected_object_image_index
                    ]
                    selected_matches = selected_objects_matches[
                        selected_object_image_index
                    ]
                    # get the target (scene) and source (object) points from the matches
                    target_points = np.array(
                        [
                            selected_scene_keypoints[match.trainIdx].pt
                            for match in selected_matches
                        ],
                        dtype=np.float32,
                    )
                    source_points = np.array(
                        [
                            selected_object_keypoints[match.queryIdx].pt
                            for match in selected_matches
                        ],
                        dtype=np.float32,
                    )

                    with LogTimer(
                        f"Calculating homography for object {selected_object_name}"
                    ):
                        homography_result = (
                            points_of_interest_impl.find_homography_ransac(
                                source_points,
                                target_points,
                                ransac_confidence_slider.value,
                                ransac_inlier_threshold_slider.value,
                            )
                        )
                        display(
                            f"Used {homography_result.num_iterations} iterations to find the homography for object {selected_object_name}"
                        )
                    homography_results.append(homography_result)

            # Display inliers
            KEY_SHOW_INLIERS_CHECKBOX = "show_inliers_checkbox"
            show_inliers_checkbox = wid.Checkbox(
                value=memoize.get(KEY_SHOW_INLIERS_CHECKBOX, default=True),
                description="Show inliers",
                **widgets_styling,
            )
            KEY_SHOW_INLIERS_OBJECT_IMAGE_DROPDOWN = (
                "show_inliers_object_image_dropdown"
            )
            show_inliers_object_image_dropdown = wid.Dropdown(
                options=object_images_selected_filenames,
                value=memoize.get(
                    KEY_SHOW_INLIERS_OBJECT_IMAGE_DROPDOWN,
                    default=object_images_selected_filenames[0],
                    possible_values=object_images_selected_filenames,
                ),
                description="Object image",
                **widgets_styling,
            )
            show_inlier_output = wid.Output()
            show_inlier_output_workaround = wid.VBox([])

            # Temporary capturing is bugged in vscode
            # @show_inlier_output.capture(
            #    clear_output=True, wait=False
            # )
            def on_show_inliers_change(change=None):
                memoize.set(KEY_SHOW_INLIERS_CHECKBOX, show_inliers_checkbox.value)
                memoize.set(
                    KEY_SHOW_INLIERS_OBJECT_IMAGE_DROPDOWN,
                    show_inliers_object_image_dropdown.value,
                )

                if show_inliers_checkbox.value:
                    show_inlier_output_workaround.children = [
                        show_inliers_object_image_dropdown
                    ]

                    selected_object_image_index = (
                        object_images_selected_filenames.index(
                            show_inliers_object_image_dropdown.value
                        )
                    )

                    # get the mask boolean array of inliers
                    inliers = homography_results[
                        selected_object_image_index
                    ].debug_inliers

                    debug_chosen_source_points = homography_results[
                        selected_object_image_index
                    ].debug_chosen_source_points

                    with LogTimer("Drawing inliers"):
                        draw_matches_params = {"matchesMask": inliers.astype(int)}
                        annotated_image = draw_matches(
                            image_1_i=object_images_selected[
                                selected_object_image_index
                            ].image_color,
                            keypoints_1=selected_objects_keypoints[
                                selected_object_image_index
                            ],
                            image_2_i=scene_image_selected.image_color,
                            keypoints_2=selected_scene_keypoints,
                            matches=selected_objects_matches[
                                selected_object_image_index
                            ],
                            params=draw_matches_params,
                        )
                        if debug_chosen_source_points is not None:
                            for source_point in debug_chosen_source_points:
                                cv2.circle(
                                    annotated_image,
                                    tuple(np.round(source_point).astype(int)),
                                    5,
                                    (255, 255, 255),
                                    -1,
                                )
                        ipy_image = to_ipy_image(
                            annotated_image,
                            longest_side=image_size * 3,
                            upscale=True,
                        )
                        # display(ipy_image)
                        show_inlier_output_workaround.children = [
                            *show_inlier_output_workaround.children,
                            ipy_image,
                        ]
                else:
                    show_inlier_output_workaround.children = []

            show_inliers_checkbox.observe(on_show_inliers_change, names="value")
            show_inliers_object_image_dropdown.observe(
                on_show_inliers_change, names="value"
            )

            display(
                wid.VBox(
                    [
                        show_inliers_checkbox,
                        show_inlier_output_workaround,
                        show_inlier_output,
                    ]
                )
            )
            on_show_inliers_change()

            # Display homography results
            annotated_scene_image = scene_image_selected.image_color.copy()
            annotated_scene_image_overlay = scene_image_selected.image_color.copy()
            for i, homography_result in enumerate(homography_results):
                if homography_result.homography is None:
                    continue

                draw_projected_object(
                    annotated_scene_image,
                    object_images_selected[i].image_color,
                    homography_result.homography,
                    draw_object=False,
                    fill_color=None,
                    outline_color=object_colors[i],
                )
                draw_projected_object(
                    annotated_scene_image_overlay,
                    object_images_selected[i].image_color,
                    homography_result.homography,
                    draw_object=True,
                    fill_color=None,
                    outline_color=object_colors[i],
                )
            display(
                to_ipy_image(
                    annotated_scene_image, longest_side=image_size * 3, upscale=True
                )
            )
            display(
                to_ipy_image(
                    annotated_scene_image_overlay,
                    longest_side=image_size * 3,
                    upscale=True,
                )
            )

        ransac_confidence_slider.observe(on_ransac_change, names="value")
        ransac_inlier_threshold_slider.observe(on_ransac_change, names="value")

        def default_ransac(change=None):
            memoize.delete_keys(
                [
                    KEY_RANSAC_CONFIDENCE_SLIDER,
                    KEY_RANSAC_INLIER_THRESHOLD_SLIDER,
                ]
            )
            on_finder_image_selection_change()

        ransac_default_button.on_click(default_ransac)

        display(
            wid.VBox(
                [
                    ransac_confidence_slider,
                    ransac_inlier_threshold_slider,
                    ransac_default_button,
                    ransac_output,
                ]
            )
        )
        on_ransac_change()

    finder_object_image_select.set_on_selection_change(on_finder_image_selection_change)
    finder_scene_image_select.set_on_selection_change(on_finder_image_selection_change)

    display(
        wid.VBox(
            [
                wid.VBox(
                    [
                        wid.HTML("<h2>Scene images</h2>"),
                        finder_scene_image_select.get_view(),
                    ]
                ),
                wid.VBox(
                    [
                        wid.HTML("<h2>Object images</h2>"),
                        finder_object_image_select.get_view(),
                    ]
                ),
                finder_image_selection_output,
            ]
        )
    )
    on_finder_image_selection_change()


finder_image_set_dropdown.observe(on_finder_menu_change, names="value")
finder_points_of_interest_impl_dropdown.observe(on_finder_menu_change, names="value")
finder_reload_impl_button.on_click(on_finder_menu_change)

display(
    wid.VBox(
        [
            wid.HBox(
                [
                    finder_image_set_dropdown,
                    finder_points_of_interest_impl_dropdown,
                    finder_reload_impl_button,
                ]
            ),
            finder_output,
        ]
    )
)
on_finder_menu_change()


2024-11-10 18:04:06.204 INFO root Displaying sift result started (notebook_cell:210)
2024-11-10 18:04:06.553 INFO root Displaying sift result took: 365.3461 ms (notebook_cell:210)


2024-11-10 18:05:12.229 INFO root Drawing inliers started (notebook_cell:395)
2024-11-10 18:05:12.581 INFO root Drawing inliers took: 345.1622 ms (notebook_cell:395)
2024-11-10 18:05:44.141 INFO root Displaying sift result started (notebook_cell:210)
2024-11-10 18:05:44.452 INFO root Displaying sift result took: 303.7845 ms (notebook_cell:210)


## TODO:
stitching